In [88]:
import django_initializer
from backend.pymongo.mongodb import db
from datetime import datetime
import arrow

# order analysis

In [89]:
cursor=db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": new_start_time
            },
            "end_at": {
                "$lte": new_end_time
            }
        }
    },
    {
        "$lookup": {
            "from": "api_order",
            "as": "orders", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                },
                { "$group": {
                    "_id": "$status",
                    "qty": {"$sum": 1}
                }}
            ]
        },
    },
    {
        "$project":{
            "_id":0,
            "campaign_id":"$id",
            "status": "$orders._id",
            "qty": "$orders.qty"
            
        }
    }
])
order = list(cursor)
display(order)

[{'campaign_id': 495, 'status': [], 'qty': []},
 {'campaign_id': 496, 'status': ['complete'], 'qty': [1]},
 {'campaign_id': 497, 'status': ['complete'], 'qty': [2]},
 {'campaign_id': 499, 'status': ['complete'], 'qty': [1]},
 {'campaign_id': 501, 'status': ['complete'], 'qty': [1]},
 {'campaign_id': 502, 'status': [], 'qty': []}]

In [116]:
class SalesReport:
    def __init__(self):
        pass
    
    @classmethod
    def normalize_start_time(cls, start_time):
        return arrow.get(start_time).replace(hour=0, minute=0, second=0).datetime
    
    @classmethod
    def normalize_end_time(cls, end_time):
        return arrow.get(end_time).replace(hour=23, minute=59, second=59).datetime
    
    @classmethod
    def get_basic_info(cls, start_time, end_time):
        query=db.api_campaign.aggregate([
            {
                "$match":{
                    "start_at":{
                        "$gte": start_time
                    },
                    "end_at": {
                        "$lte": end_time
                    }
                }
            },
            {
                "$lookup": {
                    "from": "api_campaign_comment",
                    "as": "campaign_comment",
                    'let': {'id': "$id" },
                    "pipeline":[
                        {"$match":{
                            '$expr': { '$eq': ["$$id", "$campaign_id"] },
                            "id":{"$ne":None}}
                        }
                    ]
                },
            },
            {
                "$lookup": {
                    "from": "api_campaign_product",
                    "as": "campaign_product_sold", 
                    'let': {'id': "$id" },
                    "pipeline":[
                        {
                            "$match":{
                                '$expr': { '$eq': ["$$id", "$campaign_id"] },
                                "id":{"$ne":None},
                                "qty_sold": {
                                    "$gt": 0
                                }
                            }
                        }
                    ]
                },
            },
            {
                "$lookup": {
                    "from": "api_campaign_product",
                    "as": "campaign_product_unsold", 
                    'let': {'id': "$id" },
                    "pipeline":[
                        {
                            "$match":{
                                '$expr': { '$eq': ["$$id", "$campaign_id"] },
                                "id":{"$ne":None},
                                "qty_sold": {
                                    "$eq": 0
                                }
                            }
                        }
                    ]
                },
            },
            {
                "$lookup": {
                    "from": "api_campaign_product",
                    "as": "campaign_product_total_item", 
                    'let': {'id': "$id" },
                    "pipeline":[
                        {
                            "$match":{
                                '$expr': { '$eq': ["$$id", "$campaign_id"] },
                                "id":{"$ne":None},
                            }
                        }
                    ]
                },
            },
            {
                "$lookup": {
                    "from": "api_order",
                    "as": "orders", 
                    'let': {'id': "$id" },
                    "pipeline":[
                        {
                            "$match":{
                                '$expr': { '$eq': ["$$id", "$campaign_id"] },
                                "id":{"$ne":None},
                            }
                        },
                        {
                            "$project":{
                                "_id":0,
                                "id":1,
                                "total":1
                            }
                        }
                    ]
                },
            },
            {
                "$lookup": {
                    "from": "api_pre_order",
                    "as": "pre_orders", 
                    'let': {'id': "$id" },
                    "pipeline":[
                        {
                            "$match":{
                                '$expr': { '$eq': ["$$id", "$campaign_id"] },
                                "id":{"$ne":None},
                            }
                        },
                        {
                            "$project":{
                                "_id":0,
                                "id":1,
                                "total":1
                            }
                        }
                    ]
                },
            },
            {
                "$project":{
                    "_id":0,
                    "campaign_id":"$id",
                    "post_comment":{"$size":"$campaign_comment.id"},
                    "no_of_items_sold":{"$size": "$campaign_product_sold.id"},
                    "no_of_items_unsold":{"$size": "$campaign_product_unsold.id"},
                    "total_no_of_items":{"$size": "$campaign_product_total_item.id"},
                    "total_inventories":{"$sum": "$campaign_product_total_item.qty_for_sale"},
                    "total_no_of_orders": {"$add":[{"$size": "$pre_orders.id"},{"$size": "$orders.id"}]},
                    "total_amount": {"$add":[{"$sum": "$pre_orders.total"},{"$sum": "$orders.total"}]},
                    "average_order_value":{ "$divide": [{"$add":[{"$sum": "$pre_orders.total"},{"$sum": "$orders.total"}]}, 2 ] }
                }
            }
        ])
        return list(query)
    
    @classmethod
    def get_top_10_itmes(cls, start_time, end_time):
        query=db.api_campaign.aggregate([
            {
                "$match":{
                    "start_at":{
                        "$gte": start_time
                    },
                    "end_at": {
                        "$lte": end_time
                    }
                }
            },
            {
                "$lookup": {
                    "from": "api_campaign_product",
                    "as": "campaign_product", 
                    'let': {'id': "$id" },
                    "pipeline":[
                        {
                            "$match":{
                                '$expr': { '$eq': ["$$id", "$campaign_id"] },
                                "id":{"$ne":None},
                            }
                        },
                        {
                            "$lookup": {
                                "from": "api_order_product",
                                "as": "order_product", 
                                'let': {'id': "$id" },
                                "pipeline":[
                                    {
                                        "$match":{

                                            '$expr': {"$eq": ["$$id", "$campaign_product_id"]},
                                            "id":{"$ne":None},
                                        }
                                    },
                                    {
                                        "$project":{
                                            "_id":0,
                                            "qty":1,
                                        }
                                    }
                                ]
                            },
                        },
                        {
                            "$project":{
                                "_id":0,
                                "name":1,
                                "order_product_qty":{"$sum":"$order_product.qty"},
                                "qty_for_sale":1,
                                "status": {
                                    "$switch": {
                                      "branches": [
                                        {
                                          "case": {"$eq": ["$qty_for_sale", {"$sum":"$order_product.qty"}] },
                                          "then": "Sold Out"
                                        }
                                      ],
                                      "default": ""
                                    }

                                }
                            }
                        },
                        {"$sort": {"order_product_qty":-1}},
                        { "$limit": 10 }
                    ]
                },
            },
            {
                "$project":{
                    "_id":0,
                    "campaign_id":"$id",
                    "Best selling items-Top 10": "$campaign_product.name",
                    "qty_for_sale": "$campaign_product.qty_for_sale",
                    "order_product_qty": "$campaign_product.order_product_qty",
                    "status": "$campaign_product.status"

                }
            },

        ])
        return list(query)


In [117]:
start_time = SalesReport.normalize_start_time("2022-06-15")
end_time = SalesReport.normalize_end_time("2022-06-20")
basic_info = SalesReport.get_basic_info(start_time, end_time)
top_10_itmes = SalesReport.get_top_10_itmes(start_time, end_time)

In [118]:
display(basic_info)

[{'campaign_id': 494,
  'post_comment': 50,
  'no_of_items_sold': 4,
  'no_of_items_unsold': 0,
  'total_no_of_items': 4,
  'total_inventories': 216,
  'total_no_of_orders': 16,
  'total_amount': 32280.5,
  'average_order_value': 16140.25},
 {'campaign_id': 495,
  'post_comment': 2,
  'no_of_items_sold': 1,
  'no_of_items_unsold': 2,
  'total_no_of_items': 3,
  'total_inventories': 1000,
  'total_no_of_orders': 1,
  'total_amount': 15.99,
  'average_order_value': 7.995},
 {'campaign_id': 496,
  'post_comment': 4,
  'no_of_items_sold': 4,
  'no_of_items_unsold': 2,
  'total_no_of_items': 6,
  'total_inventories': 1050,
  'total_no_of_orders': 3,
  'total_amount': 159.9,
  'average_order_value': 79.95},
 {'campaign_id': 497,
  'post_comment': 14,
  'no_of_items_sold': 1,
  'no_of_items_unsold': 1,
  'total_no_of_items': 2,
  'total_inventories': 115,
  'total_no_of_orders': 4,
  'total_amount': 40.0,
  'average_order_value': 20.0},
 {'campaign_id': 499,
  'post_comment': 13,
  'no_of_ite

In [119]:
display(top_10_itmes)

[{'campaign_id': 494,
  'Best selling items-Top 10': ['Vegetable Box A',
   'GSI Cooking Set',
   'MSR 2person Tent',
   'RC2 - Test'],
  'qty_for_sale': [200, 10, 5, 1],
  'order_product_qty': [27, 5, 5, 1],
  'status': ['', '', 'Sold Out', 'Sold Out']},
 {'campaign_id': 495,
  'Best selling items-Top 10': ['Top', 'Jeans', 'Skirt'],
  'qty_for_sale': [200, 500, 300],
  'order_product_qty': [1, 0, 0],
  'status': ['', '', '']},
 {'campaign_id': 496,
  'Best selling items-Top 10': ['grapegfruit',
   'Pepsi',
   'Skirt',
   'Thai Meow Tea',
   'prize yummy cookies',
   'Prize - Bag'],
  'qty_for_sale': [200, 300, 400, 100, 40, 10],
  'order_product_qty': [6, 2, 2, 1, 0, 0],
  'status': ['', '', '', '', '', '']},
 {'campaign_id': 497,
  'Best selling items-Top 10': ['nina0615', 'nina0615'],
  'qty_for_sale': [100, 15],
  'order_product_qty': [3, 0],
  'status': ['', '']},
 {'campaign_id': 499,
  'Best selling items-Top 10': ['cc013',
   'cecitest12',
   'cecitest5gift',
   'cecitest6'],
 